# Part 1 - Introduction to Hive
Hive provides an interface for querying and managing large datasets in HDFS. Hive can be used to project structure onto large datasets already residing in distributed storage. Once a structure is defined, we can use the same querying language we used against a MySQL database (SQL)to generate insights on the existing data.

One of the benefits of Hive is that it abstracts the complexity of large-scale parallel processing on datasets. Users can use a familiar query language (SQL) that they may already have experience in using on smaller scale data on local machines, and Hive will abstract the same query across (potentially) thousands of machines and terabytes of data.

In this prac we will look at applying structure to the HR Dataset flat file we uploaded to HDFS in prac 2 so that we can query it using SQL.


## Starting Hive
We will first need to SSH into the data7001 node to access the Hive command line interface.

Like in Prac 2, use the Terminal program to SSH into clientnode.zones.eait.uq.edu.au, using your sXXXXXX username as the username.

You can begin Hive by typing "hive" into the command line prompt. It may take a few seconds to initialise. When it's ready to receive commands, you should see a prompt like the following:

```
hive>
```

Hive allows you to think of your files in HDFS as a database, and query it in a similar way you would in MySQL. In order to create new tables, you will first need to connect to your database. Connect to your database using the following command, where the `sXXXXXX` is replaced with your student number. 

```
hive> use sXXXXXXX;
```

If working, Hive should return an "OK" message along with how long the query took to make.

Now that we are working within the correct database, the next step will be to load the HR Dataset we previously pushed into HDFS into a temporary table, called `tmp_hr`. First we will need to create the temporary table using the following syntax:

```
create table tmp_hr (col_value STRING);
```

Once we've created the table, we can import our HR Analytics dataset from HDFS into our newly created table using the following command, replacing sXXXXXX with your student number:

```
LOAD DATA INPATH '/user/sXXXXXX/HR_comma_sep.csv' OVERWRITE INTO TABLE tmp_hr;
```

You may have noticed that the above statement doesn't include any information about the column types in our HR Analytics dataset. We first create a temporary table to *consume* the data into Hive. The temporary table contains only one column which is treated as a string. As you would have learned from last weeks prac, we are limited with what we can do with just a string data type. Let's create a more structured table that informs Hive about what each columns data type is in our HR Analytics dataset.

First, we need to create a new table that has the appropriate structure. We need to know what the column headers are, as well as the data types for those columns. Enter the following command into Hive to create a new table called `hr` with the appropriate data types.

```
CREATE TABLE HR (satisfaction_level FLOAT, last_evaluation FLOAT, number_project INT, average_monthly_hours INT, time_spend_company INT, left_job INT, promotion_last_5_years INT, sales STRING, salary STRING);
```

You may notice a new data type in the above statement, a "FLOAT". A float is short for a floating point number, these refer to a data type that include a decimal point - it is derived from the fact that there are no fixed number of digits before and after the decimal point. For example, the value "1" can be represented as an INT, or as the FLOAT "1.0". Programs will deal with floating point numbers differently than they do integers, so it is important to make sure you select the right data type.

Now we have a new empty table with the required structure, but we still need to import the data from our temporary table into our new table. Enter in the following command to extract the appropriate parts from the `tmp_hr` table and insert them into our structured table.
```
INSERT OVERWRITE TABLE hr
SELECT
  regexp_extract(col_value, '^(?:([^,]*),?){1}', 1) satisfaction_level,
  regexp_extract(col_value, '^(?:([^,]*),?){2}', 1) last_evaluation,
  regexp_extract(col_value, '^(?:([^,]*),?){3}', 1) number_project,
  regexp_extract(col_value, '^(?:([^,]*),?){4}', 1) average_monthly_hours,
  regexp_extract(col_value, '^(?:([^,]*),?){5}', 1) time_spend_company,
  regexp_extract(col_value, '^(?:([^,]*),?){6}', 1) left_job,
  regexp_extract(col_value, '^(?:([^,]*),?){7}', 1) promotion_last_5_years,
  regexp_extract(col_value, '^(?:([^,]*),?){8}', 1) sales,
  regexp_extract(col_value, '^(?:([^,]*),?){9}', 1) salary
FROM tmp_hr;		
```


You may not have come across the `regexp_extract` syntax in SQL before. Let's break the above statement down a little bit. Remember that our `tmp_hr` table does not contain structure, each row is just treated as a long comma separated string, but our intention is to extract each word between the commas. How can we do this? So far we have had software handle the complexities of parsing comma separated values, but how could we otherwise extract values between strings?

One method is using something called a *regular expression*. A regular expression is a sequence of characters that describe a search pattern, and are used heavily in computer science. Although understanding them is not in the scope of this course, they are worth learning as they are a powerful tool in finding patterns and cleaning data. The following website provides a great starting point to understanding how these work:

https://regexone.com/

In our above statement, the regular expression is the string "`(?:([^,]*),?){1}`, as you may have guessed, this statement matches any strings between the comma character. The `{1}` refers to the nth match that is used to fill the specified column. For example: `regexp_extract(col_value, '^(?:([^,]*),?){1}', 1) satisfaction_level` extracts the first match of the regular expression, and places it into the `satisfaction_level` column.

Now that we have ingested our data into a new, structured table called `hr`, we can query the table exactly like we did in phpMyAdmin, using SQL. As an interface, you will see that Hive is very similar to a traditional database. However, instead of using a database backend, Hive can abstract our queries over many machines. Although not obvious in a dataset this size, if we had a dataset that was several terabytes in size, you would see significant performance gain over a tradition database as it is backed by a distributed file system and the queries can be distributed across several machines. 

Answer the following questions and **include the SQL query used to determine the answer**.

Unlike phpMyAdmin, you will need to ensure that you end each statement with a ';'.

|<center>TASK</center>|
| ---- |
| How many entries were there in the HR dataset? |

Answer:
15000

Code:
hive> SELECT COUNT(*)
    > FROM hr;
[Enter your answere here! Include the SQL query used. ]

|<center>TASK</center>|
| ---- |
| What was the average number of monthly hours? |

Answer:
201.0503366891126

Code:
hive> SELECT AVG(average_monthly_hours) AS AVG_MONTHLY_HOURS
    > FROM hr;
[ Enter your answere here! Include the SQL query used. ]

|<center>TASK</center>|
| ---- |
| What was the average number of monthly hours by those who left their job? |

Answer:
199.81834946980175

Code:
hive> SELECT AVG(average_monthly_hours) AS AVG_MONTHLY_HOURS
    > FROM hr
    > WHERE left_job=1;
[ Enter your answer here! Include the SQL query used. ]

Unlike MySQL, you can see that some of your queries were submitted as *jobs*, and may have taken a few seconds to process. Jobs are a common way of describing how processing is submitted to large distributed systems. Unlike your phpMyAdmin database that was only used by you, large distributed systems are often shared and processing may take hours or months. This means that often a job has to be submitted, picked up by some sort of job management process, directed to the appropriate node/s, distribute processing across multiple jobs, maintain a job queue and collate the results across multiple nodes (among many other things!). You can see why this complexity adds time - especially since many of these interactions are over a network. The benefits of a distributed system are only apparent when your dataset is big enough to warrant using one. In our case of our small HR Dataset, it is actually much faster to use R or MySQL!